# SIngleton Pattern
1. Comes under Creational Pattern
1. Used to control access to Class instances
1. And to ensure that the class has only on instance
1. Provides a global point of access
1. The class itself is responsible for creating the instance
1. Supports Lazy Instantiation
1. Reduces the global namespace
1. Subclassible for extended use
1. Variable number of instances possible with - Base Class and Meta Class variants
1. More flexible than static class - math - has no instance

> **Usefull for** Controlling access to limited resources
> - Device Access
> - Buffer pools
> - Web/DB connection pools
> - Lazy Instantiation for heavy service

> Use it sparingly! as it's an **Antipattern**

In [1]:
class Singleton():
    ans = 'Meh'
    
    @staticmethod
    def instance():
        if '_instance' not in Singleton.__dict__:
            Singleton._instance = Singleton()
        return Singleton._instance

In [2]:
s1 = Singleton.instance()
s2 = Singleton.instance()

s1 is s2

True

In [3]:
s1.ans = 'One Instance to rule them all!'

s2.ans

'One Instance to rule them all!'

# Logging Subsystem
1. Logs events to a file
1. Only one instance can write to the file
1. Need to control access

# Classic Singleton

In [4]:
import datetime

In [5]:
class Logger():
    log_file = None
    
    @staticmethod
    def instance():
        if '_instance' not in Logger.__dict__:
            Logger._instance = Logger()
        return Logger._instance
    
    def open_log(self, path):
        self.log_file = open(path, mode='w')
        
    def write_log(self, log_record):
        now = datetime.datetime.now()
        record = f'{now}: {log_record}'
        self.log_file.writelines(record)
        
    def close_log(self):
        self.log_file.close()

In [6]:
logger = Logger.instance()
logger.open_log('../output/test.log')
logger.write_log('Writing Logs using classic Singleton pattern!\n')
logger.write_log('Usefull for Debug Purpose!')
logger.close_log()

In [7]:
with open('../output/test.log', 'r') as file:
    for line in file:
        print(line, end='')

2017-12-15 01:55:33.289831: Writing Logs using classic Singleton pattern!
2017-12-15 01:55:33.289831: Usefull for Debug Purpose!

# Problems with Singleton - Antipattern
1. Violates Single Responsibility - Look after their own instantiation then hold and process that state
1. Non standard class access
1. Harder to test
1. Carry global state
1. Hard to sub class
1. Tight coupling with driver program
1. Maintenance Issue

# Solution
Although - below issues still remain
1. Testing Issue
1. Global State
1. Maintenance Issue

# Modified Singleton

In [8]:
class Singleton:
    _instances = {}
    
    def __new__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__new__(cls)
            cls._instances[cls] = instance
            
        return cls._instances[cls]

In [9]:
class BetterLogger(Singleton):
    log_file = None
    
    def __init__(self, path):
        if self.log_file is None:
            self.log_file = open(path, mode='w')
        
    def write_log(self, log_record):
        now = datetime.datetime.now()
        record = f'{now}: {log_record}'
        self.log_file.writelines(record)
        
    def close_log(self):
        self.log_file.close()

In [10]:
logger = BetterLogger('../output/test.log')
logger.write_log('Writing Logs using Better Singleton pattern!\n')
logger_2 = BetterLogger('../output/this-file-name-will-be-ignored.log')
logger.write_log('It solves many problems inherent with Singleton')
logger.close_log()

In [11]:
with open('../output/test.log', 'r') as file:
    for line in file:
        print(line, end='')

2017-12-15 01:55:33.368593: Writing Logs using Better Singleton pattern!
2017-12-15 01:55:33.368593: It solves many problems inherent with Singleton

# Even Better Singleton
Using Metaclass
1. A class is an instance of a Metaclass
1. Metaclass controls the building of a class

In [12]:
class Singleton(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
            
        return cls._instances[cls]

In [13]:
class EvenBetterLogger(metaclass=Singleton):
    log_file = None
    
    def __init__(self, path):
        if self.log_file is None:
            self.log_file = open(path, mode='w')
        
    def write_log(self, log_record):
        now = datetime.datetime.now()
        record = f'{now}: {log_record}'
        self.log_file.writelines(record)
        
    def close_log(self):
        self.log_file.close()

In [14]:
logger = EvenBetterLogger('../output/test.log')
logger.write_log('Writing Logs using Even Better Singleton pattern!\n')
logger_2 = EvenBetterLogger('../output/this-file-name-will-be-ignored.log')
logger.write_log('It solves the separation of concern problem')
logger.close_log()

In [15]:
with open('../output/test.log', 'r') as file:
    for line in file:
        print(line, end='')

2017-12-15 01:55:33.443589: Writing Logs using Even Better Singleton pattern!
2017-12-15 01:55:33.443589: It solves the separation of concern problem

# MonoState Pattern
1. It's okay with global state
1. It takes advantage of python's dynamic nature
1. Usefull for multiple instances, but we want them to share same state

In [16]:
class MonoState():
    _state = {}
    
    def __new__(cls, *args, **kwargs):
        self = super().__new__(cls)
        self.__dict__ = cls._state
        
        return self

In [17]:
class BestLogger(MonoState):
    log_file = None
    
    def __init__(self, path):
        if self.log_file is None:
            self.log_file = open(path, mode='w')
        
    def write_log(self, log_record):
        now = datetime.datetime.now()
        record = f'{now}: {log_record}'
        self.log_file.writelines(record)
        
    def close_log(self):
        self.log_file.close()

In [18]:
logger = BestLogger('../output/test.log')
logger.write_log('Writing Logs using Mono State!\n')
logger_2 = BestLogger('../output/this-file-name-will-be-ignored.log')
logger.write_log("It's okay with a global state!")
logger.close_log()

In [19]:
with open('../output/test.log', 'r') as file:
    for line in file:
        print(line, end='')

2017-12-15 01:55:33.517588: Writing Logs using Mono State!
2017-12-15 01:55:33.517588: It's okay with a global state!